# Building Semantic Memory with Embeddings

So far, we've mostly been treating the kernel as a stateless orchestration engine.
We send text into a model API and receive text out. 

In a [previous notebook](04-kernel-arguments-chat.ipynb), we used `kernel arguments` to pass in additional
text into prompts to enrich them with more data. This allowed us to create a basic chat experience. 

However, if you solely relied on kernel arguments, you would quickly realize that eventually your prompt
would grow so large that you would run into a the model's token limit. What we need is a way to persist state
and build both short-term and long-term memory to empower even more intelligent applications. 

To do this, we dive into the key concept of `Semantic Memory` in the Semantic Kernel. 

In [4]:
#r "nuget: Microsoft.SemanticKernel, 1.2.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.2.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Sqlite, 1.2.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.2.0 Microsoft.SemanticKernel.Connectors.Sqlite, 1.2.0-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.2.0-alpha System.Linq.Async, 6.0.1

In [5]:
using System.IO;
string indexFile = "index.db";
if (File.Exists(indexFile))
    File.Delete(indexFile);

In [6]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.Sqlite;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Memory functionality is experimental
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0052, SKEXP0028

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var index = !File.Exists("index.db");
var store = await SqliteMemoryStore.ConnectAsync("index.db");

var memoryBuilder = new MemoryBuilder();

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
        "text-embedding-ada-002",
        azureEndpoint, 
        apiKey,
        "model-id");
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);
}


memoryBuilder.WithMemoryStore(store);

var memory = memoryBuilder.Build();

In [7]:
using System;
using System.Collections.Generic;
using System.Text.Json;
using System.Text.Json.Serialization;

public class Cocktail
{
    public string Name { get; set; }
    public string Glass { get; set; }
    public string Category { get; set; }
    public List<IngredientItem> Ingredients { get; set; }
    public string Garnish { get; set; }
    public string Preparation { get; set; }
}

public class IngredientItem
{
    public string Unit { get; set; }
    public decimal? Amount { get; set; }
    public string Ingredient { get; set; }
    public string Label { get; set; }
    public string Special { get; set; }
}


In [8]:
var jsonFileContents = File.ReadAllText("data/recipes.json");
var options = new JsonSerializerOptions
        {
            PropertyNameCaseInsensitive = true
        };
var cocktails = System.Text.Json.JsonSerializer.Deserialize<List<Cocktail>>(jsonFileContents, options);
cocktails

index value 0 Submission#9+Cocktail Name Vesper Glass martini Category Before Dinner Cocktail Ingredients index value 0 Submission#9+IngredientItem Unit cl Amount 6 Ingredient Gin Label <null> Special <null> 1 Submission#9+IngredientItem Unit cl Amount 1.5 Ingredient Vodka Label <null> Special <null> 2 Submission#9+IngredientItem Unit cl Amount 0.75 Ingredient Lillet Blonde Label <null> Special <null> Garnish Lemon twist Preparation Shake and strain into a chilled cocktail glass. 1 Submission#9+Cocktail Name Bacardi Glass martini Category Before Dinner Cocktail Ingredients index value 0 Submission#9+IngredientItem Unit cl Amount 4.5 Ingredient White rum Label Bacardi White Rum Special <null> 1 Submission#9+IngredientItem Unit cl Amount 2 Ingredient Lime juice Label <null> Special <null> 2 Submission#9+IngredientItem Unit cl Amount 1 Ingredient Syrup Label Grenadine Special <null> Garnish <null> Preparation Shake with ice cubes. Strain into chilled cocktail glass. 2 Submission#9+Cocktail Name Negroni Glass old-fashioned Category Before Dinner Cocktail Ingredients index value 0 Submission#9+IngredientItem Unit cl Amount 3 Ingredient Gin Label <null> Special <null> 1 Submission#9+IngredientItem Unit cl Amount 3 Ingredient Campari Label <null> Special <null> 2 Submission#9+IngredientItem Unit cl Amount 3 Ingredient Vermouth Label Sweet red vermouth Special <null> Garnish Half an orange slice Preparation Build into old-fashioned glass filled with ice. Stir gently. 3 Submission#9+Cocktail Name Rose Glass martini Category <null> Ingredients index value 0 Submission#9+IngredientItem Unit cl Amount 2 Ingredient Kirsch Label <null> Special <null> 1 Submission#9+IngredientItem Unit cl Amount 4 Ingredient Vermouth Label Dry vermouth Special <null> 2 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special 3 dashes Strawberry syrup Garnish <null> Preparation Stir all ingredients with ice and strain into a cocktail glass. 4 Submission#9+Cocktail Name Old Fashioned Glass old-fashioned Category Before Dinner Cocktail Ingredients index value 0 Submission#9+IngredientItem Unit cl Amount 4.5 Ingredient Whiskey Label Bourbon or rye whiskey Special <null> 1 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special 2 dashes Angostura Bitters 2 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special 1 sugar cube 3 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special Few dashes plain water Garnish Orange slice and cherry Preparation Place sugar cube in old-fashioned glass and saturate with bitters, add a dash of plain water. Muddle until dissolve. Fill the glass with ice cubes and add whisky. 5 Submission#9+Cocktail Name Tuxedo Glass martini Category All Day Cocktail Ingredients index value 0 Submission#9+IngredientItem Unit cl Amount 3 Ingredient Gin Label Old Tom Gin Special <null> 1 Submission#9+IngredientItem Unit cl Amount 3 Ingredient Vermouth Label Dry vermouth Special <null> 2 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special 1/2 bar spoon Maraschino 3 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special 1/4 bar spoon Absinthe 4 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special 3 dashes Orange Bitters Garnish Cherry and lemon twist Preparation Stir all ingredients with ice and strain into cocktail glass. 6 Submission#9+Cocktail Name Mojito Glass collins Category Longdrink Ingredients index value 0 Submission#9+IngredientItem Unit cl Amount 4 Ingredient White rum Label White Cuban Rum Special <null> 1 Submission#9+IngredientItem Unit cl Amount 3 Ingredient Lime juice Label <null> Special <null> 2 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> Label <null> Special 6 Mint sprigs 3 Submission#9+IngredientItem Unit <null> Amount <null> Ingredient <null> 

In [9]:
public string FlattenCocktail(Cocktail cocktail)
{
    var ingredients = cocktail.Ingredients.Select(ingredient =>
    {
        var ingredientDesc = $"{ingredient.Amount} {ingredient.Unit} {ingredient.Ingredient}";
        if (!string.IsNullOrWhiteSpace(ingredient.Label))
        {
            ingredientDesc += $" ({ingredient.Label})";
        }
        if (!string.IsNullOrWhiteSpace(ingredient.Special))
        {
            ingredientDesc += $", Special: {ingredient.Special}";
        }
        return ingredientDesc;
    });

    string ingredientsList = string.Join(", ", ingredients);
    string cocktailDescription = $"{cocktail.Name}, Glass: {cocktail.Glass}, Category: {cocktail.Category}, Ingredients: {ingredientsList}, Garnish: {cocktail.Garnish}, Preparation: {cocktail.Preparation}";

    return cocktailDescription;
}

In [10]:
using Microsoft.SemanticKernel.Text;

const string MemoryCollectionName = "CocktailCollection";

// Memory functionality is experimental
#pragma warning disable SKEXP0055

// For each learning process the chunks
foreach(var cocktail in cocktails)
{
    // Break the learnings into paragraphs
    //var result = TextChunker.SplitPlainTextLines(FlattenCocktail(cocktail), 100);

    await memory.SaveInformationAsync(MemoryCollectionName, id: cocktail.Name, text: FlattenCocktail(cocktail));
};
